https://dataxujing.github.io/CNN-paper2/#/zh-cn/chapter10?id=%e5%9b%be%e5%83%8f%e5%88%86%e5%89%b2

## 图像分割中几种定义

* 语义分割(Semantic Segmentation)：就是对一张图像上的所有像素点进行分类(eg: **FCN/Unet/Unet++/...**)，但语义分割只能判断类别，无法区分个体。
* 实例分割(Instance Segmentation)：可以理解为目标检测和语义分割的结合(eg: **Mask R-CNN/...**)，它不需要对每个像素进行标记，它只需要找到感兴趣物体的边缘轮廓就行。
 >相对目标检测的边界框，实例分割可精确到物体的边缘

 >相对语义分割，实例分割需要标注出图上同一物体的不同个体
* 全景分割(Panoptic Segmentation)：可以理解为语义分割和实例分割的结合
 >实例分割只对图像中的object进行检测，并对检测到的object进行分割

 >全景分割是对图中的所有物体包括背景都要进行检测和分割

![1-1](../../img/1-1.png)

>图像分类：图像中的气球是一个类别。

>语义分割：分割出气球和背景。

>目标检测：图像中有7个目标气球，并且检测出每个气球的坐标位置。

>实例分割：图像中有7个不同的气球，在像素层面给出属于每个气球的像素。

图像分割相关论文：https://github.com/mrgloom/awesome-semantic-segmentation

FCN、Unet、DeepLab、SegNet、PSPNet

# FCN网络的理解

[论文地址](https://link.zhihu.com/?target=https%3A//arxiv.org/abs/1411.4038)

![FCN1](../../img/FCN1.png)
FCN将一般的经典的分类网络模型（VGG16...）的最后一层的FC层（全连接）换成**卷积**，这样可以通过二维的特征图，后接softmax获得**每个像素点的分类信息**，从而解决了分割问题。
> 核心思想：
> - 不含全连接层(fc)的全卷积(fully conv)网络。**可适应任意尺寸输入。**
> - 增大数据尺寸的反卷积(deconv)层。**能够输出精细的结果。**
> - 结合不同深度层结果的跳级(skip)结构。**同时确保鲁棒性和精确性。**
> - 损失函数是在最后一层的 spatial map上的 pixel 的 loss 和，在每一个 pixel 使用 softmax loss

![FCN结构示意图](../../img/1-3.jpg)

> **对于FCN-32s**，直接对pool5 feature进行32倍上采样获得32x upsampled feature，再对32x upsampled feature每个点做softmax prediction获得32x upsampled feature prediction（即分割图）。

> **对于FCN-16s**，首先对pool5 feature进行2倍上采样获得2x upsampled feature，再把pool4 feature和2x upsampled feature逐点相加，然后对相加的feature进行16倍上采样，并softmax prediction，获得16x upsampled feature prediction。

> **对于FCN-8s**，首先进行pool4+2x upsampled feature逐点相加，然后又进行pool3+2x upsampled逐点相加，即进行更多次特征融合。

![FCN-32s;FCN-16s;FCN-8s，也就是：使用多层feature融合有利于提高分割准确性。](../../img/FCN2.png)

**FCN优点**

1. 实现端到端分割
2. 将深的粗糙的信息与浅的精细的信息相结合，产生准确和精细的分割
3. FCN可以接受任意尺寸的输入图像。采用反卷积层对最后一个卷基层的特征图（feature map）进行上采样，使它恢复到输入图像相同的尺寸，从而可以对每一个像素都产生一个预测，同时保留了原始输入图像中的空间信息

**FCN缺点**
1. 结果不够精细。进行8倍上采样虽然比32倍的效果好了很多，但是上采样的结果还是比较模糊和平滑，**对图像中的细节不敏感。**
2. 对各个像素进行分类，**没有充分考虑像素与像素之间的关系**。忽略了在通常的基于像素分类的分割方法中使用的空间规整（spatial regularization）步骤，缺乏空间一致性。



# U-Net网络的理解

[论文地址](https://arxiv.org/pdf/1505.04597.pdf)

![U-net结构示意图](../../img/U-Net.png)

> 核心思想：
> - 首先进行Conv+Pooling下采样；
> - 然后反卷积进行上采样，crop之前的低层feature map，进行融合；
> - 再次上采样。
> - 重复这个过程，直到获得输出388x388x2的feature map，
> - 最后经过softmax获得output segment map。总体来说与FCN思路非常类似。

**UNet的encoder下采样4次，一共下采样16倍，对称地，其decoder也相应上采样4次，将encoder得到的高级语义特征图恢复到原图片的分辨率。**

它采用了与FCN不同的*特征融合*方式：

* FCN采用的是**逐点相加**，对应tensorflow的tf.add()函数
* U-Net采用的是**channel维度拼接融合**，对应tensorflow的tf.concat()函数

**U-Net优点**
1. 同时具备捕捉上下文信息的收缩路径和允许精确定位的对称扩展路径，这使得网络将上下文信息向更高层分辨率传播
2. Overlap-tile 策略，这种方法用于补全输入图像的上下信息，可以解决由于内存不足造成的图像输入的问题
3. 同时使用弹性形变对数据进行增强，既可以解决训练数据不足的问题，也可以提高算法的鲁棒性
4. 使用了加权损失。预先计算权重图，一方面补偿了训练数据每类像素的不同频率，另一方面使网络更注重学习相互接触的细胞间边缘。

**U-Net缺点**

并不适合所有的分割任务，比如对刚体进行分割，因为很难通过弹性形变对刚体数据进行增强。

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class double_conv2d_bn(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=3,strides=1,padding=1):
        super(double_conv2d_bn,self).__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,
                              kernel_size=kernel_size,
                              stride=strides,padding=padding,bias=True)
        self.conv2 = nn.Conv2d(out_channels,out_channels,
                              kernel_size=kernel_size,
                              stride=strides,padding=padding,bias=True)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out
    
class deconv2d_bn(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=2,strides=2):
        super(deconv2d_bn,self).__init__()
        self.conv1 = nn.ConvTranspose2d(in_channels,out_channels,
                                        kernel_size = kernel_size,
                                       stride = strides,bias=True)
        self.bn1 = nn.BatchNorm2d(out_channels)
       
     def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        return out

class Unet(nn.Module):
    def __init__(self):
        
    def forward(self,x):
        

# UNet++

UNet++对Unet改进的点主要是**skip connection**，作者认为skip connection 直接将unet中encoder的浅层特征与decoder的深层特征结合是不妥当的，会产生semantic gap。

文中假设：当所结合的浅层特征与深层特征是**semantically similar**时，网络的优化问题就会更简单，因此文章对skip connection的改进就是**bridge/reduce 这个semantic gap**。

![UNet++结构](../../img/1-6.jpg)

[论文地址](https://arxiv.org/pdf/1807.10165.pdf)

[代码地址](https://github.com/MrGiovanni/UNetPlusPlus)

# Unet+++

为了弥补UNet和UNet++的缺陷，UNet+++中的每一个解码器层都融合了来自**编码器中的小尺度和同尺度的特征图**，以及来自**解码器的大尺度的特征图**，这些特征图捕获了**全尺度下的细粒度语义和粗粒度语义**。

![UNet+++结构](./img/1-7.jpg)

[论文地址](https://arxiv.org/ftp/arxiv/papers/2004/2004.08790.pdf)

# DeepLab v3+算法简阅

![DeepLab v3+结构](../../img/1-8.jpg)

**Encoder部分**
> Encoder就是原来的DeepLabv3，
需要注意点：
> 1. 输入尺寸与输出尺寸比（output stride = 16），最后一个stage的膨胀率rate为2

> 2. Atrous Spatial Pyramid Pooling module（ASPP）有四个不同的rate，额外一个全局平均池化

**Decoder部分**
> 先把encoder的结果上采样4倍，然后与resnet中下采样前的Conv2特征concat一起，再进行3x3的卷积，最后上采样4倍得到最终结果。

> 需要注意点：融合低层次信息前，先进行1x1的卷积，目的是降通道（例如有512个通道，而encoder结果只有256个通道）

[论文地址](https://arxiv.org/pdf/1802.02611.pdf)

# Unet在医学图像上的适用与CNN分割算法的简要总结

1. Unet结构特点

> UNet相比于FCN和Deeplab等，共进行了4次上采样，并在同一个stage使用了skip connection，而不是直接在高级语义特征上进行监督和loss反传，这样就保证了最后恢复出来的特征图融合了更多的low-level的feature，也使得不同scale的feature得到了的融合，从而可以进行多尺度预测和DeepSupervision。4次上采样也使得分割图恢复边缘等信息更加精细。

2. 为什么适用于医学图像？

> 因为医学图像边界模糊、梯度复杂，需要较多的高分辨率信息。高分辨率用于精准分割。

> 人体内部结构相对固定，分割目标在人体图像中的分布很具有规律，语义简单明确，低分辨率信息能够提供这一信息，用于目标物体的识别。

> UNet结合了**低分辨率信息**（提供物体类别识别依据）和**高分辨率信息**（提供精准分割定位依据），完美适用于医学图像分割。

3. 分割算法改进总结：

> **下采样+上采样**：Convlution + Deconvlution／Resize
> **多尺度特征融合**：特征逐点相加／特征channel维度拼接
> **获得像素级别的segement map**：对每一个像素点进行判断类别

4.[UNet-family Github](https://github.com/ShawnBIT/UNet-family)